In [ ]:
import pandas as pd
import geopandas as gpd
import osmnx as ox
import folium
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Define the place of interest (POI)

In [ ]:
place = "Manhattan, United States"
graph = ox.graph_from_place(place, network_type="drive")

With the above two lines of code, OSMnx allows us to retrieve the street networks of the city quickly. The result is a Networkx class, and we convert it to Geopandas dataframe to further process the data.

In [ ]:
len(graph)

# 2. Convert the data type

In [ ]:
nodes, streets = ox.graph_to_gdfs(graph)
streets.head()

Now the graph has been converted to the familiar form of a pandas data frame with extra capability for Geographic data handling.
Use Seanborn to do some basic visualization

# 3. Data Visualization

In [ ]:
#street
street_type = pd.DataFrame(streets["highway"].apply(pd.Series)[0].value_counts().reset_index())
street_type.columns=["type", "count"]

fig, ax = plt.subplots(figsize=(12,10))

sns.barplot(y="type", x="count", data=street_type, ax=ax)
plt.tight_layout()


In [ ]:
style = {'color': '#F7DC6F', 'weight':'1'}
m = folium.Map([40.730610, -73.935242],
               zoom_start=15,
               tiles="CartoDb dark_matter")

folium.GeoJson(streets, style_function=lambda x: style).add_to(m)
m.save("streets.html")
m

In [ ]:
#Building Footprints
bldg = ox.footprints_from_place(place)
bldg.shape

In [ ]:
cols = ['amenity', 'building', 'name', 'tourism']
bldg[cols].head()

In [ ]:
bldg_type = pd.DataFrame(bldg["amenity"].apply(pd.Series)[0].value_counts().reset_index())

bldg_type.columns=["type", "count"]

fig, ax = plt.subplots(figsize=(12,10))

sns.barplot(y="type", x="count", data=bldg_type, ax=ax)
plt.tight_layout()

In [ ]:
style_buildings = {'color':'#6C3483 ', 'fillColor': '#6C3483 ', 'weight':'1', 'fillOpacity' : 1}

b = folium.Map([40.730610, -73.935242],
               zoom_start=15,
               tiles="Stamen Toner")

folium.GeoJson(bldg[:1000], style_function=lambda x: style_buildings).add_to(b)
b.save("buildings.html")
b

In [ ]:
#Cafe https://wiki.openstreetmap.org/wiki/Key:amenity

cafe = ox.pois_from_place(place, amenities = ["cafe"])
cafe.head()

In [ ]:
cafe_pt = cafe[cafe.geom_type == "Point"]

In [ ]:
c = folium.Map([40.730610, -73.935242], 
               zoom_start=10, 
               tiles="CartoDb dark_matter")
locs = zip(cafe_pt.geometry.y, cafe_pt.geometry.x)
#folium.GeoJson(buildings, style_function=lambda x: style_buildings).add_to(m)
for location in locs:
    folium.CircleMarker(location=location, 
        color = "#F4F6F7",   radius=2).add_to(c)
c.save("cafes.html")
c